In [1]:
import torch
import torch.nn as nn

from src.models import notMIWAE

In [2]:
encoder = nn.Sequential(nn.Linear(1000,50), nn.ReLU(), nn.Linear(50,10))
decoder = nn.Sequential(nn.Linear(10,50), nn.ReLU(), nn.Linear(50,1000))
missing_model = nn.Sequential(nn.Linear(1000, 1000), nn.Sigmoid())

In [3]:
model = notMIWAE(encoder, decoder, missing_model, 1000, 10, 1000, 10)

In [4]:
x = torch.randn(64, 1000)
s = torch.randint(0, 2, (64, 1000)).float()

In [5]:
model(x, s, K = 2)

tensor(1913.0227, grad_fn=<NegBackward0>)